In [16]:
import numpy as np
import cv2
import os
from skimage.restoration import denoise_nl_means, estimate_sigma
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [17]:
def leer_imagenes(folder, target_size=(32, 32)):
    images = []
    # Get the total number of images in the folder
    num_images = len(os.listdir(folder))
    # Initialize tqdm with total number of images
    pbar = tqdm(total=num_images, desc='Processing Images')

    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img_resized = cv2.resize(img, target_size)
            images.append(img_resized)
            # Update the progress bar
            pbar.update(1)

    # Close the progress bar
    pbar.close()
    return images

In [18]:
def convertir_a_grises(images):
    pbar = tqdm(total=len(images), desc='Converting to Gray')
    # gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    # gray_images = [np.expand_dims(img, axis=-1) for img in gray_images]
    gray_images = []

    for img in images:
        temp_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        temp_img = np.expand_dims(temp_img, axis=-1)
        gray_images.append(temp_img)
        pbar.update(1)
    pbar.close()
    return gray_images


In [19]:
from skimage.restoration import denoise_nl_means, estimate_sigma

def aplicar_filtro_anisotropico(images):
    pbar = tqdm(total=len(images), desc='Aplying filter')
    filtered_images = []
    for image in images:
        # Asegúrate de que image tiene la forma esperada, o ajusta el channel_axis
        # Si las imágenes son (altura, anchura, 1), usa channel_axis=-1
        sigma_est = np.mean(estimate_sigma(image, channel_axis=-1))
        denoised = denoise_nl_means(image, h=1.15 * sigma_est, fast_mode=True,
                                    patch_size=5, patch_distance=6)
        filtered_images.append(denoised)
        pbar.update(1)
    pbar.close()
    
    return filtered_images


In [20]:
def generar_muestras(images, filtered_images, k=32):
    pbar = tqdm(total=len(images), desc='Generate Mustras')
    x_list = []
    y_list = []
    for img, filtered_img in zip(images, filtered_images):

        # x, y = np.random.randint(0, img.shape[0] - k), np.random.randint(0, img.shape[1] - k)
        original_patch = img #[x:x+k, y:y+k]
        filtered_patch = filtered_img #[x:x+k, y:y+k]
        x_list.append(original_patch)
        y_list.append(filtered_patch)
        pbar.update(1)
    pbar.close()
    return np.array(x_list), np.array(y_list)  


In [21]:
def main(folder='test', k=32):
    images = leer_imagenes(folder)
    gray_images = convertir_a_grises(images)
    filtered_images = aplicar_filtro_anisotropico(gray_images)
    x, y = generar_muestras(gray_images, filtered_images, k=k)
    return x, y

# Ejecutar la función main
x, y = main()
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Generate Mustras: 100%|██████████| 200/200 [00:00<?, ?it/s]


In [22]:
from UNet import UNet
input_shape = (32, 32, 1)
unet = UNet()
model = unet.build_model(input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    x=X_train, 
    y=y_train, 
    epochs=1, 
    validation_data=(x_test, y_test)
)

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32, 32, 32), output.shape=(32, 16, 16, 3)